In [7]:
import spacy
import pickle
import re
import logging

spacy_model = spacy.load("en_core_web_sm")
topics ={
    0: 'bank account services',
    1: 'credit_card',
    2: 'mortgage/loans',
    3: 'theft/dispute reporting',
    4: 'others'
}

logistic_model, tf_idf = None, None
with open('../src/model/logistic_model.pkl','rb') as f:
    logistic_model = pickle.load(f)

with open('../src/model/tf_idf.pkl','rb') as f:
    tf_idf = pickle.load(f)

### Prediction pipeline

In [8]:
def pre_process_text(text):
	text = text.lower()
	text = re.sub(r'\[.*?\]','',text)
	text = re.sub(r'[^\w\s]', '',text)
	text = re.sub(r'[\d]','',text)
	text = re.sub(r'[*x]?','',text)
	print(f"1. pre-processed: {text}")
	return text

def get_lemma(text):
	"""This function returns only noun words"""
	modified_text =""
	tokens = spacy_model(text)
	for token in tokens :
		if(token.tag_=='NN'):
			modified_text += token.lemma_ +" "

	print(f"2. lemmatization: {modified_text}")
	return modified_text

def get_tf_idf_features(text):
	print("3. tf_idf transformation")

	X = tf_idf.transform([text])
	return X.toarray()

def get_predict(X):
	y = logistic_model.predict(X)
	cls = topics.get(y[0])
	print(f'4. prediction: "{cls}" class')
	return cls

def predict_class(text):
	print(f"0. Input Text: {text}")
	preprocessedText = pre_process_text(text)
	lemmatizedText = get_lemma(preprocessedText)

	X = get_tf_idf_features(lemmatizedText)
	category = get_predict(X)
	return category

In [10]:
output = predict_class("I have a problem with my credit card")

0. Input Text: I have a problem with my credit card
1. pre-processed: i have a problem with my credit card
2. lemmatization: problem credit card 
3. tf_idf transformation
4. prediction: "credit_card" class


In [11]:
output

'credit_card'

## API Call

In [39]:
import requests

In [70]:
# passing a sample payload
payload = {
    "text": """
        I have transferred money from my account!!!!
        But the transaction history doesn't seem to reflect it @#%$$#~
        ...Where did my money go?
        Please check and let me know asap.
        I have attached the screenshot of the transaction.
    """
}

# calling the predict endpoint 
# Please note that the flask application has to be running
response = requests.post("http://127.0.0.1:5000/predict", json=payload)
response

<Response [200]>

In [71]:
response.json()

{'prediction': 'bank account services'}